# Dogs vs Cats(CNNを用いた犬猫判定)

In [6]:
import os, shutil
from keras import callbacks
from keras import optimizers
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pickle
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.applications import VGG16

#元のデータセットを展開したディレクトリへのパス
original_dataset_dir = "./original/train"

#より小さなデータセットを格納するディレクトリへのパス
base_dir = "./cats_and_dogs_small"
os.mkdir(base_dir)

#訓練、検証、テストデータセットを配置するディレクトリ
train_dir = os.path.join(base_dir, "train")
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, "validation")
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, "test")
os.mkdir(test_dir)

#訓練の猫の画像
train_cats_dir = os.path.join(train_dir, "cats")
os.mkdir(train_cats_dir)
#訓練dogs
train_dogs_dir = os.path.join(train_dir, "dogs")
os.mkdir(train_dogs_dir)
#検証cats
validation_cats_dir = os.path.join(validation_dir, "cats")
os.mkdir(validation_cats_dir)
#検証dogs
validation_dogs_dir = os.path.join(validation_dir, "dogs")
os.mkdir(validation_dogs_dir)
#テストcats
test_cats_dir = os.path.join(test_dir, "cats")
os.mkdir(test_cats_dir)
#テストdogs
test_dogs_dir = os.path.join(test_dir, "dogs")
os.mkdir(test_dogs_dir)

#最初の1000個の猫画像をtrain_cats_dirにコピー
fnames = ["cat.{}.jpg".format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)
#次の500個の猫をvalidation_cats_dirにコピー
fnames = ["cat.{}.jpg".format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
#次の500個の猫をtest_cats_dirにコピー
fnames = ["cat.{}.jpg".format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)

#最初の1000個のいぬ画像をtrain_cats_dirにコピー
fnames = ["dog.{}.jpg".format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
#次の500個のいぬをvalidation_cats_dirにコピー
fnames = ["dog.{}.jpg".format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
#次の500個のいぬをtest_cats_dirにコピー
fnames = ["dog.{}.jpg".format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [7]:
print("train_cats:", len(os.listdir(train_cats_dir)))
print("train_dogs:", len(os.listdir(train_dogs_dir)))
print("validation_cats:", len(os.listdir(validation_cats_dir)))
print("validation_dogs:", len(os.listdir(validation_dogs_dir)))
print("test_cats:", len(os.listdir(test_cats_dir)))
print("test_dogs:", len(os.listdir(test_dogs_dir)))

train_cats: 1000
train_dogs: 1000
validation_cats: 500
validation_dogs: 500
test_cats: 500
test_dogs: 500


In [8]:
import os, shutil
from keras import callbacks
from keras import optimizers
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pickle
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.applications import VGG16

#より小さなデータセットを格納するディレクトリへのパス
base_dir = "./cats_and_dogs_small"

#訓練、検証、テストデータセットを配置するディレクトリ
train_dir = os.path.join(base_dir, "train")

validation_dir = os.path.join(base_dir, "validation")

test_dir = os.path.join(base_dir, "test")


#訓練の猫の画像
train_cats_dir = os.path.join(train_dir, "cats")

#訓練dogs
train_dogs_dir = os.path.join(train_dir, "dogs")

#検証cats
validation_cats_dir = os.path.join(validation_dir, "cats")

#検証dogs
validation_dogs_dir = os.path.join(validation_dir, "dogs")

#テストcats
test_cats_dir = os.path.join(test_dir, "cats")

#テストdogs
test_dogs_dir = os.path.join(test_dir, "dogs")

In [9]:
print("train_cats:", len(os.listdir(train_cats_dir)))
print("train_dogs:", len(os.listdir(train_dogs_dir)))
print("validation_cats:", len(os.listdir(validation_cats_dir)))
print("validation_dogs:", len(os.listdir(validation_dogs_dir)))
print("test_cats:", len(os.listdir(test_cats_dir)))
print("test_dogs:", len(os.listdir(test_dogs_dir)))

train_cats: 1000
train_dogs: 1000
validation_cats: 500
validation_dogs: 500
test_cats: 500
test_dogs: 500


# 学習済みCNNを使用(特徴抽出)

In [10]:
conv_base = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))

conv_base.summary()

W0918 14:47:22.500289 18480 deprecation_wrapper.py:119] From c:\users\haruk\anaconda3\envs\dev\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0918 14:47:22.537222 18480 deprecation_wrapper.py:119] From c:\users\haruk\anaconda3\envs\dev\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0918 14:47:22.544189 18480 deprecation_wrapper.py:119] From c:\users\haruk\anaconda3\envs\dev\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0918 14:47:22.581100 18480 deprecation_wrapper.py:119] From c:\users\haruk\anaconda3\envs\dev\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0918 14:47:22.876975 18480 deprecation_wra

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

# データ拡張を行う特徴抽出

In [11]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


# モデルの訓練の前に

In [12]:
print('This is the number of trainable weights '
      'before freezing the conv base:', len(model.trainable_weights))

conv_base.trainable = False

print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 30
This is the number of trainable weights after freezing the conv base: 4


# 拡張してモデル全体を訓練

In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

# 検証データは水増しすべきでないことに注意
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=20, class_mode='binary') 
validation_generator = test_datagen.flow_from_directory(validation_dir,target_size=(150, 150),batch_size=20,class_mode='binary')

model.compile(loss="binary_crossentropy", optimizer=optimizers.RMSprop(lr=2e-5), metrics=["acc"])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

W0918 14:49:07.339319 18480 deprecation_wrapper.py:119] From c:\users\haruk\anaconda3\envs\dev\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0918 14:49:07.346301 18480 deprecation.py:323] From c:\users\haruk\anaconda3\envs\dev\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/30
 - 20s - loss: 0.5989 - acc: 0.6855 - val_loss: 0.4658 - val_acc: 0.8150
Epoch 2/30
 - 17s - loss: 0.4881 - acc: 0.7755 - val_loss: 0.3736 - val_acc: 0.8600
Epoch 3/30
 - 17s - loss: 0.4388 - acc: 0.8010 - val_loss: 0.3292 - val_acc: 0.8820
Epoch 4/30
 - 17s - loss: 0.4021 - acc: 0.8205 - val_loss: 0.3313 - val_acc: 0.8620
Epoch 5/30
 - 17s - loss: 0.3821 - acc: 0.8310 - val_loss: 0.2976 - val_acc: 0.8920
Epoch 6/30
 - 17s - loss: 0.3649 - acc: 0.8330 - val_loss: 0.2795 - val_acc: 0.8910
Epoch 7/30
 - 17s - loss: 0.3542 - acc: 0.8510 - val_loss: 0.2777 - val_acc: 0.8980
Epoch 8/30
 - 17s - loss: 0.3565 - acc: 0.8460 - val_loss: 0.2717 - val_acc: 0.8890
Epoch 9/30
 - 17s - loss: 0.3434 - acc: 0.8470 - val_loss: 0.2651 - val_acc: 0.8990
Epoch 10/30
 - 17s - loss: 0.3383 - acc: 0.8560 - val_loss: 0.2654 - val_acc: 0.9010
Epoch 11/30
 - 17s - loss: 0.3215 - acc: 0.8575 - val_loss: 0.2575 - val_ac

In [14]:
model.save('cats_and_dogs_small.h5')
with open("history.pickle", mode="wb") as f:
    pickle.dump(history.history, f)

In [17]:
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150, 150), batch_size=20, class_mode="binary")

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)

print("test acc:", test_acc)

Found 1000 images belonging to 2 classes.
test acc: 0.902999997138977


# 提出用ファイルの作成

In [18]:
img = [0]*12500
x_test = [0]*12500
for i in range(12499):
    # filenameのUserNameの部分は変更してください。
    filename = "./original/test/" + str(i+1) + ".jpg"
    img[i] = image.load_img(filename, target_size=(150, 150))
    x_test[i] = image.img_to_array(img[i])
    x_test[i] = np.expand_dims(x_test[i], axis=0)
    x_test[i] = x_test[i]/255.0

In [19]:
model = models.load_model("cats_and_dogs_small.h5")
with open("history.pickle", mode="rb") as f:
    history = pickle.load(f)

c:\users\haruk\anaconda3\envs\dev\lib\site-packages\keras\engine\saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [20]:
import pandas as pd
expect = [0]*12500
for i in range(12499):
     expect[i]= model.predict(x_test[i], batch_size = 20 ).flatten().tolist()[0]

id =[]
for i in range(1, 12501):
    id.append(i)

submission_DC = pd.DataFrame({"Id" : id, "label" : expect})
submission_DC.to_csv("submission_DC.csv", index=False)